In [2]:
import pandas as pd
import numpy as np


import torch
from numba import cuda
cuda.select_device(0)
cuda.close()
cuda.select_device(0)


f = "/home/nikolal/projects/parlaspeech/transcripts_normalized.txt"
df = pd.read_table(f, names=["unknown1", "path", "sentence"],
        #nrows=2000
        )

df["path"] = "/home/peterr/macocu/task6_speech/data/" + \
    df.path.str.replace("flac/", "") + ".wav"
df = df.drop(columns="unknown1")
import os
df["filesize"] = df["path"].apply(os.path.getsize)
for char in ['*', ':', ';']:
    df["sentence"] = df.sentence.str.replace(char, "")
    
df.head()

,path,sentence,filesize
0,/home/peterr/macocu/task6_speech/data/00000001...,poštovane kolegice i kolege zastupnici molio b...,249678
1,/home/peterr/macocu/task6_speech/data/00000002...,prvi klub koji se javio za stanku je klub esde...,118478
2,/home/peterr/macocu/task6_speech/data/00000003...,u njihovo ime predstavnik bojan glavašević,96078
3,/home/peterr/macocu/task6_speech/data/00000004...,hvala gospodine predsjedniče,38478
4,/home/peterr/macocu/task6_speech/data/00000005...,poštovani gospodine predsjedniče hrvatskog sab...,368078


In [3]:
from sklearn.model_selection import train_test_split

common_voice_train_df, common_voice_test_df = train_test_split(df, train_size=0.8, )


#common_voice_train_df.reset_index(drop=True, inplace=True)
common_voice_test_df.reset_index(drop=True, inplace=True)

import datasets
from datasets import load_dataset, load_metric, Audio
def load_audio(path):
    return datasets.Audio(sampling_rate=16000).decode_example(path)

#common_voice_train_df.loc[:, "audio"] = common_voice_train_df.path.apply(load_audio)
common_voice_test_df.loc[:, "audio"] = common_voice_test_df.path.apply(load_audio)

#common_voice_train_dataset = datasets.Dataset.from_pandas(common_voice_train_df)
common_voice_test_dataset = datasets.Dataset.from_pandas(common_voice_test_df)


from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(
    "./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(
    feature_extractor=feature_extractor, tokenizer=tokenizer)





from transformers import Trainer
from transformers import TrainingArguments
from transformers import Wav2Vec2ForCTC
from typing import Any, Dict, List, Optional, Union
from dataclasses import dataclass, field
import torch


def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(
        audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch


#common_voice_train_mapped = common_voice_train_dataset.map(
    #prepare_dataset, remove_columns=common_voice_train_dataset.column_names)
common_voice_test_mapped = common_voice_test_dataset.map(
    prepare_dataset, remove_columns=common_voice_test_dataset.column_names)

/home/peterr/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/peterr/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
file ./config.json not found
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/4608 [00:00<?, ?ex/s]

In [4]:
repo_name = "3_new_attempt/checkpoint-2000"

# model = Wav2Vec2ForCTC.from_pretrained(
#     repo_name,
#     attention_dropout=0.0,
#     hidden_dropout=0.0,
#     feat_proj_dropout=0.0,
#     mask_time_prob=0.05,
#     layerdrop=0.0,
#     ctc_loss_reduction="mean",
#     pad_token_id=processor.tokenizer.pad_token_id,
#     vocab_size=len(processor.tokenizer),
# )

# processor = Wav2Vec2Processor.from_pretrained(repo_name)
model = Wav2Vec2ForCTC.from_pretrained(repo_name).to("cuda")


tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(
    "./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
processor = Wav2Vec2Processor(
    feature_extractor=feature_extractor, tokenizer=tokenizer)
     


file ./config.json not found
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
#rezs = list()
for i in common_voice_test_df.index:
    try:
        input_dict = processor(common_voice_test_mapped[i]["input_values"], return_tensors="pt", padding=True,
        sampling_rate=16000)
        logits = model(input_dict.input_values.to("cuda")).logits
        pred_ids = torch.argmax(logits, dim=-1)[0]

        gold = common_voice_test_df.sentence[i]   
        prediction = processor.decode(pred_ids)
        fullpath = common_voice_test_df.path[i]
        filename = fullpath.split("/")[-1]
        rezs.append({"file": filename, "gold": gold, "transcribed": prediction})
    finally:

        import json
        with open("results_3_new_attempt.json", "w") as f:
            json.dump(rezs, f)


RuntimeError: CUDA out of memory. Tried to allocate 50.00 MiB (GPU 0; 31.75 GiB total capacity; 30.16 GiB already allocated; 6.75 MiB free; 30.54 GiB reserved in total by PyTorch)

In [7]:
import json
with open("results_3_new_attempt.json", "w") as f:
    json.dump(rezs, f)